In [34]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [4]:
%ls datasets

normal_json/  output/  rotate_json/


In [42]:
df = pd.read_json('datasets/output/annotations/instances_train.json', lines=True)

In [43]:
df

,info,licenses,images,categories,annotation
0,"{'year': '2020', 'version': '1.0', 'descriptio...","[{'url': 'xxxx'}, {'name': 'xxxxxx'}]","[{'filename': '1_130_90_0_04077.jpg', 'height'...","[{'supercategory': 'Person', 'id': 0, 'name': ...","[{'segmentation': [[264, 319, 274, 319, 274, 3..."


In [45]:
df['annotation'][0]

[{'segmentation': [[264, 319, 274, 319, 274, 331, 264, 331]],
  'area': 120,
  'image_id': 0,
  'bbox': [264, 319, 10, 12],
  'category_id': 1,
  'id': 0},
 {'segmentation': [[342, 295, 353, 295, 353, 308, 342, 308]],
  'area': 143,
  'image_id': 0,
  'bbox': [342, 295, 11, 13],
  'category_id': 1,
  'id': 1},
 {'segmentation': [[330, 304, 341, 304, 341, 316, 330, 316]],
  'area': 132,
  'image_id': 0,
  'bbox': [330, 304, 11, 12],
  'category_id': 1,
  'id': 2},
 {'segmentation': [[319, 328, 331, 328, 331, 343, 319, 343]],
  'area': 180,
  'image_id': 0,
  'bbox': [319, 328, 12, 15],
  'category_id': 1,
  'id': 3},
 {'segmentation': [[325, 343, 339, 343, 339, 359, 325, 359]],
  'area': 224,
  'image_id': 0,
  'bbox': [325, 343, 14, 16],
  'category_id': 1,
  'id': 4},
 {'segmentation': [[317, 358, 335, 358, 335, 370, 317, 370]],
  'area': 216,
  'image_id': 0,
  'bbox': [317, 358, 18, 12],
  'category_id': 1,
  'id': 5},
 {'segmentation': [[343, 336, 377, 336, 377, 394, 343, 394]],
  

# Load the data  and json file into a dataframe with proper keys

In [46]:
annot = pd.DataFrame(df['annotation'][0])
img = pd.DataFrame(df['images'][0])

In [47]:
annot[['x min', 'x max', 'y_min', 'y_max']]  = pd.DataFrame(annot['bbox'].tolist(), index=annot.index)

In [48]:
s = annot.merge(img.drop(['height', 'width'], axis=1), left_on='image_id', right_on='id')

In [49]:
s

,segmentation,area,image_id,bbox,category_id,id_x,xmin,xmax,y_min,y_max,filename,id_y
0,"[[264, 319, 274, 319, 274, 331, 264, 331]]",120,0,"[264, 319, 10, 12]",1,0,264,319,10,12,1_130_90_0_04077.jpg,0
1,"[[342, 295, 353, 295, 353, 308, 342, 308]]",143,0,"[342, 295, 11, 13]",1,1,342,295,11,13,1_130_90_0_04077.jpg,0
2,"[[330, 304, 341, 304, 341, 316, 330, 316]]",132,0,"[330, 304, 11, 12]",1,2,330,304,11,12,1_130_90_0_04077.jpg,0
3,"[[319, 328, 331, 328, 331, 343, 319, 343]]",180,0,"[319, 328, 12, 15]",1,3,319,328,12,15,1_130_90_0_04077.jpg,0
4,"[[325, 343, 339, 343, 339, 359, 325, 359]]",224,0,"[325, 343, 14, 16]",1,4,325,343,14,16,1_130_90_0_04077.jpg,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17623,"[[321, 381, 342, 381, 342, 417, 321, 417]]",756,2026,"[321, 381, 21, 36]",0,17623,321,381,21,36,1_60_60_0_00418.jpg,2026
17624,"[[300, 295, 312, 295, 312, 330, 300, 330]]",420,2027,"[300, 295, 12, 35]",0,17624,300,295,12,35,1_60_60_0_00433.jpg,2027
17625,"[[622, 14, 634, 14, 634, 37, 622, 37]]",276,2027,"[622, 14, 12, 23]",0,17625,622,14,12,23,1_60_60_0_00433.jpg,2027
17626,"[[575, 366, 594, 366, 594, 392, 575, 392]]",494,2027,"[575, 366, 19, 26]",0,17626,575,366,19,26,1_60_60_0_00433.jpg,2027


In [35]:
root = Path(os.getcwd())
train_annot_path = root / "datasets" / "output" / "train" / "txt_annotations"
train_annot_path.exists()

In [119]:
c = 0
for img_id in img['id']:
    subset = s.loc[s['image_id'] == img_id, ['category_id', 'xmin', 'xmax', 'y_min', 'y_max', 'filename']].reset_index(drop=True)
    try:
        filename = train_annot_path / subset.at[0, 'filename']
        filename = filename.with_suffix('.txt')
        subset.drop('filename', axis=1).to_csv(filename, header=None, index=False)
    except KeyError as e:
        print("Image ID not found in annotations: ", img_id)
        c += 1

print("Total images not found: ", c)

Image ID not found in annotations:  111
Image ID not found in annotations:  115
Image ID not found in annotations:  316
Image ID not found in annotations:  427
Image ID not found in annotations:  529
Image ID not found in annotations:  604
Image ID not found in annotations:  623
Image ID not found in annotations:  858
Image ID not found in annotations:  879
Image ID not found in annotations:  969
Image ID not found in annotations:  1080
Image ID not found in annotations:  1138
Image ID not found in annotations:  1215
Image ID not found in annotations:  1263
Image ID not found in annotations:  1292
Image ID not found in annotations:  1333
Image ID not found in annotations:  1489
Image ID not found in annotations:  1557
Image ID not found in annotations:  1561
Image ID not found in annotations:  1567
Image ID not found in annotations:  1998
Total images not found:  21


In [110]:
subset = s.loc[s['image_id'] == 111, ['category_id', 'xmin', 'xmax', 'y_min', 'y_max', 'filename']].reset_index(drop=True)

,segmentation,area,image_id,bbox,category_id,id_x,xmin,xmax,y_min,y_max,filename,id_y


False